In [1]:
!pip install datefinder

You should consider upgrading via the 'c:\users\loris\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datefinder
from datetime import datetime
import collections
import json
import os
from tqdm.auto import tqdm
import asyncio
import aiohttp
from aiohttp import  ClientResponseError
import re

data_path = f"data"
result_path = f"models/"

scrape_identifier = "fullfact_2023-13-07"

result_path_fullfact = f"models/{scrape_identifier}/"
result_path_fullfact_partial_lists = f"models/{scrape_identifier}/lists/"
result_path_fullfact_partial_fact_checks = f"models/{scrape_identifier}/fact_checks/"
result_path_fullfact_partial_speakers = f"models/{scrape_identifier}/speakers/"
result_path_fullfact_partial_metadata = f"models/{scrape_identifier}/metadata/"
os.makedirs(result_path, exist_ok=True)
os.makedirs(result_path_fullfact, exist_ok=True)
os.makedirs(result_path_fullfact_partial_lists, exist_ok=True)

fullfact_pages_number = 513


def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()


def load_json(p):
    if os.path.exists(p):
        with open(p, "r", encoding="latin1") as f:
            d = json.load(f)
        return d
    else:
        return {}

usa_states_names = load_json(f"{data_path}united_states_names.json")

pol_labels_map = {
    'pants-on-fire': 0,
    'lie': 0,
    'false': 1,
    'barely-true': 2,
    'mostly-false': 2,
    'half-true': 3,
    'mostly-true': 4,
    'true': 5,
    'full-flop': 6,
    'half-flip': 7,
    'no-flip': 8
}
#VALORI TROVATI FACENDO SET DELLE fact_check_ground_truth_label ESTRATTE
# partly false
# false headline
# true
# opinion
# prank generator
# false
# satire
# mixture
# missing context
# altered

c:\Users\loris\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Metodi di supporto

def split_string(input_string):
    pattern = r'\.(?!\w)'
    results = re.split(pattern, input_string)

    results_without_newline = [result.replace("\n", "") for result in results]
    return results_without_newline

def extract_truthfulness_and_motivation(input_string):
    split_results = split_string(input_string)
    pattern = r'rated this claim as\s*<a href="([^"]+)">([^<]+)</a>'
    for i, result in enumerate(split_results):
        match = re.search(pattern, result)
        if match:
            motivation = result.split('</a>')[1]
            content = match.group(2)
            # Rimozione gli spazi vuoti multipli
            motivation = ' '.join(motivation.split())
            return content, motivation
    return None, None

def find_date_string(date):
    date_parsed = datefinder.find_dates(date)
    for date_current in date_parsed:
        date_string = str(date_current)
        return date_string


def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.abc.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [11]:
#@title Download dalla pagina Lastest di FullFact
df_pages = []

for index_page in tqdm(range(1, fullfact_pages_number)):
    fullfact_page_path = f"{result_path_fullfact_partial_lists}page_{index_page}.csv"

    if not os.path.exists(fullfact_page_path):
        page_data = []
        page_url = f"https://fullfact.org/latest/?page={index_page}"
        webpage = requests.get(page_url)
        page = BeautifulSoup(webpage.content, 'html.parser')
        fact_check_boxes = page.find_all('div', class_='row no-gutters')

        for fact_check_container in fact_check_boxes:
            fact_check_date = fact_check_container.find('div', class_='published-at').text.strip()
            fact_check_date = datetime.strptime(fact_check_date,"%d %B %Y").strftime("%Y-%m-%d")
            fact_check_description = fact_check_container.find('p', class_='card-text').text.strip()
            fact_check_link = fact_check_container.find('a', class_='stretched-link')
            thumbnail = fact_check_container.find('img', class_='card-img')
            fact_check_image_url_thumbnail = f"https://www.fullfact.org{thumbnail['src']}" if thumbnail else ""
            thumbnail = fact_check_image_url_thumbnail if pd.notnull(fact_check_image_url_thumbnail) else ""

            statement_title = fact_check_container.find('h3').text.strip()
           
            fact_check_url = f"https://www.fullfact.org{fact_check_link['href']}"
            fact_check_id = fact_check_url.split('/')[-2]

            page_data.append({
                'page_index': index_page,
                'fact_check_id': fact_check_id,
                'fact_check_explanation_title': statement_title,
                'fact_check_explanation_summary': fact_check_description,
                'fact_check_image_url_thumbnail': thumbnail,
                'fact_check_date': fact_check_date,
                'fact_check_page_url': fact_check_url
            })

        page_df = pd.DataFrame(page_data)
        page_df.to_csv(fullfact_page_path, index=False)

  0%|          | 0/512 [00:00<?, ?it/s]

100%|██████████| 512/512 [00:00<00:00, 13682.25it/s]


# 2 - Costruzione unico dataframe con tutte le url

In [12]:
df_pages = []
for index_page in tqdm(range(1, fullfact_pages_number)):
    fullfact_page_path = f"{result_path_fullfact_partial_lists}page_{index_page}.csv"
    page_data = pd.read_csv(fullfact_page_path)
    df_pages.append(page_data)
page_data_full = pd.concat(df_pages)
page_data_full.reset_index(drop=True, inplace=True)
page_data_full.to_csv(f"{result_path_fullfact}{scrape_identifier}_fact_checks_list.csv", index=False)
display(page_data_full)

100%|██████████| 512/512 [00:00<00:00, 620.83it/s]


,page_index,fact_check_id,fact_check_explanation_title,fact_check_explanation_summary,fact_check_image_url_thumbnail,fact_check_date,fact_check_page_url
0,1,video-Alexa-predicts-WWIII-fake,Amazon virtual assistant Alexa does not predic...,"In a viral audio clip, Alexa supposedly predic...",https://www.fullfact.org/media/_versions/alexa...,2023-07-14,https://www.fullfact.org/online/video-Alexa-pr...
1,1,World-Cup-ball-investigated-remote-control,No investigation into football used in 2022 Wo...,Multiple posts on social media claim an invest...,https://www.fullfact.org/media/_versions/stadi...,2023-07-14,https://www.fullfact.org/online/World-Cup-ball...
2,1,missing-girl-false-US-photo,Missing girl hoax uses photo of kidnapped US c...,The hoax post appeals for help to find a girl ...,https://www.fullfact.org/media/_versions/faceb...,2023-07-14,https://www.fullfact.org/online/missing-girl-f...
3,1,tax-revenue-Brexit,The UK’s tax revenue isn’t £40 billion less th...,A now-corrected article in The Guardian fails ...,https://www.fullfact.org/media/_versions/tax_r...,2023-07-13,https://www.fullfact.org/economy/tax-revenue-B...
4,1,HIV-AIDS-smallpox-covid-fauci-patient-zero,False claims of links between vaccines and HIV...,A social media post falsely links Dr Anthony F...,https://www.fullfact.org/media/_versions/dr._a...,2023-07-12,https://www.fullfact.org/health/HIV-AIDS-small...
...,...,...,...,...,...,...,...
6135,512,legg-passes-judgement-expenses,Legg passes judgement on expenses,Sir Thomas Legg today published his long await...,NaN,2010-02-04,https://www.fullfact.org/news/legg-passes-judg...
6136,512,free-personal-care-what-price,Free personal care - at what price?,The Labour Government has committed to introdu...,NaN,2010-02-01,https://www.fullfact.org/news/free-personal-ca...
6137,512,new-controversy-battle-over-future-climate-pol...,New controversy in battle over the future of c...,Climate change controversy increased today as ...,NaN,2010-01-28,https://www.fullfact.org/news/new-controversy-...
6138,512,trust-and-engagement-british-politics-continue...,Trust and engagement in British politics conti...,The British Social Attitudes survey today repo...,NaN,2010-01-26,https://www.fullfact.org/news/trust-and-engage...


# 3 - Download efficiente del contenuto di ciascuna pagina di fact-check

In [13]:
os.makedirs(result_path_fullfact_partial_fact_checks, exist_ok=True)
os.makedirs(result_path_fullfact_partial_metadata, exist_ok=True)

# Initialize connection pool
conn = aiohttp.TCPConnector(limit_per_host=100, ttl_dns_cache=300)
PARALLEL_REQUESTS = 10
tasks = []
errors = []
results = []

df_scrape = pd.DataFrame(columns=[
    'fact_check_id',
    'fact_check_page_url',
    'fact_check_page_timestamp_fetch',
    'fact_check_page_status_code',
    'fact_check_page_encoding',
    'fact_check_page_content_type',
    'fact_check_page_error_message'
])

async def gather_with_concurrency(n):
    semaphore = asyncio.Semaphore(n)
    session = aiohttp.ClientSession(connector=conn, trust_env=True)

    def build_response_dict(id, url, type, data, body=None):
        return {
            'id': id,
            'url': url,
            'type': type,
            'data': data,
            'body': body
        }

    # heres the logic for the generator
    async def get(row):
        page_id = row['fact_check_id']
        page_url = row['fact_check_page_url']
        async with semaphore:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61/63 Safari/537.36 Edg/100.0.1185.39',
                'Accept-Encoding': 'gzip'
            }
            try:
                async with session.get(page_url, headers=headers, raise_for_status=True, timeout=30, ssl=False) as resp:
                    if 'text/html' in resp.content_type or 'text/plain' or 'json' in resp.content_type:
                        return build_response_dict(page_id, page_url, 'data', resp, await resp.text())
                    else:
                        print(page_url)
                        print(resp.content_type)
                        print(resp.content_disposition)
                        assert False
            except asyncio.TimeoutError as error:
                error_code = 'timeout_error'
                return build_response_dict(page_id, page_url, 'error', error, error_code)
            except ClientResponseError as error:
                error_code = 'client_response_error'
                return build_response_dict(page_id, page_url, 'error', error, error_code)

    print("Generating asynchronous requests")
    for index, row in tqdm(page_data_full.iterrows(), total=page_data_full.shape[0]):
        if not os.path.exists(f"{result_path_fullfact_partial_metadata}{row['fact_check_id']}.json"):
            tasks.append(asyncio.create_task(get(row)))
        else:
            metadata = load_json(f"{result_path_fullfact_partial_metadata}{row['fact_check_id']}.json")
            df_scrape.loc[len(df_scrape)] = metadata

    print("Processing asynchronous requests")
    for request_current in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        result = await request_current
        result_id = result['id']
        result_url = result['url']
        result_type = result['type']
        result_data = result['data']
        result_body = result['body']

        timestamp_now = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
        row = {
            'fact_check_id': result_id,
            'fact_check_page_url': result_url,
            'fact_check_page_timestamp_fetch': find_date_string(timestamp_now)
        }

        if result_type == 'error':
            row['fact_check_page_status_code'] = result_data.status
            row['fact_check_page_encoding'] = None
            row['fact_check_page_content_type'] = None
            row['fact_check_page_error_message'] = result_data.message
            df_scrape.loc[len(df_scrape)] = row
        else:
            row['fact_check_page_status_code'] = result_data.status
            try:
                row['fact_check_page_encoding'] = result_data.get_encoding().lower() if result_data.get_encoding() else None
            except RuntimeError:
                row['fact_check_page_encoding'] = None
            headers = flatten(result_data.headers)
            if 'Content-Type' in headers.keys():
                row['fact_check_page_content_type'] = headers['Content-Type']
                page_source_path = f"{result_path_fullfact_partial_fact_checks}{row['fact_check_id']}.html"
                if 'text/html' in row['fact_check_page_content_type']:
                    with open(page_source_path, 'w', encoding=row['fact_check_page_encoding']) as f:
                        f.write(result_body)
                    serialize_json(f"{result_path_fullfact_partial_metadata}", f"{row['fact_check_id']}.json", row)
                    df_scrape.loc[len(df_scrape)] = row
                else:
                    raise f"Unexpected content type found: {row['fact_check_page_content_type']}"

    await session.close()

await gather_with_concurrency(PARALLEL_REQUESTS)
conn.close()

df_scrape.to_csv(f"{result_path_fullfact}{scrape_identifier}_scrape_data.csv", index=False)

Generating asynchronous requests


100%|██████████| 6140/6140 [00:01<00:00, 6136.07it/s]


Processing asynchronous requests


100%|██████████| 6140/6140 [07:50<00:00, 13.04it/s]


In [38]:
df_scrape = pd.read_csv(f"{result_path_fullfact}{scrape_identifier}_scrape_data.csv")
df_summary = pd.read_csv(f"{result_path_fullfact}{scrape_identifier}_fact_checks_list.csv")

dump_data = {}
# if os.path.exists(f"{result_path_fullfact}{scrape_identifier}_fact_checks.json"):
#     dump_data = load_json(f"{result_path_fullfact}{scrape_identifier}_fact_checks.json")

##Download correzzioni

In [40]:
corrections_data = []
page_url = f"https://fullfact.org/about/corrections/"
webpage = requests.get(page_url)
page = BeautifulSoup(webpage.content, 'html.parser')
list_corrections = page.find_all('li', class_='list-group-item')

for correction in list_corrections:
    date = correction.find('strong').text.strip()
    date = datetime.strptime(date, "%d %b %Y").strftime("%Y-%m-%d")
    for strong in correction('strong'):
        strong.decompose()
    description = correction.get_text(separator='', strip=True)
    description = description.replace('<a>', '').replace('</a>', '').replace('\n', '')

    link = correction.find('a')
    id = link.get('href').split('/')[-2]
    ids = link.get('href').split('/')
    complete_link = 'https://fullfact.org'+link.get('href')
    corrections_data.append({
        'fact_check_id': id,
        'correction_description': description,
        'fact_checker_correction_date':date,
        'complete_link': complete_link,
    })
corrections_df = pd.DataFrame(corrections_data)


corrections_df.to_csv(f"{result_path_fullfact}{scrape_identifier}_corrections.csv", index=False)
display(corrections_df)

,fact_check_id,correction_description,fact_checker_correction_date,complete_link
0,police-track-location-999-call,Police can’t automatically track your location...,2023-06-21,https://fullfact.org/online/police-track-locat...
1,who-covid-pandemic-over,The WHO has not declared the Covid-19 pandemic...,2023-06-16,https://fullfact.org/health/who-covid-pandemic...
2,martin-lewis-national-insurance-government,Government website makes percentage error on N...,2023-05-17,https://fullfact.org/news/martin-lewis-nationa...
3,babies-water,Advice to ‘NEVER’ give babies water is missing...,2023-04-25,https://fullfact.org/health/babies-water/
4,kate-shemirani-cause-death,The UK is not being flooded with contaminated ...,2023-04-25,https://fullfact.org/health/kate-shemirani-cau...
...,...,...,...,...
88,why-poll-gives-misleading-view-how-many-people...,Why this poll gives a misleading view on how m...,2020-08-19,https://fullfact.org/health/why-poll-gives-mis...
89,have-200000-nurses-quit-nhs,"Have 200,000 nurses quit the NHS?The date of t...",2020-08-14,https://fullfact.org/health/have-200000-nurses...
90,coronavirus-death-count-data,Changes to government Covid-19 death data won’...,2020-07-28,https://fullfact.org/health/coronavirus-death-...
91,coronavirus-social-distancing-rules,What are the new lockdown rules on social dist...,2020-07-24,https://fullfact.org/health/coronavirus-social...


# 4 - Logica di parsing per estrarre le informazioni salienti da ciascuna pagina

In [42]:
from IPython.display import display
for index, row in tqdm(df_scrape.iterrows(), total=df_scrape.shape[0]):

    fact_check_id = row['fact_check_id']
    fact_check_url = row['fact_check_page_url']

    #Queste info non sono presenti nella singola pagina, quindi in caso si fa la join con _fact_checks_list.csv
    summary_row = df_summary[df_summary['fact_check_id'] == fact_check_id]
    fact_check_explanation_title = summary_row['fact_check_explanation_title'].values[0]
    fact_check_explanation_summary = summary_row['fact_check_explanation_summary'].values[0]
    fact_check_image_url_thumbnail = "" if pd.isna(summary_row['fact_check_image_url_thumbnail'].values[0]) else summary_row['fact_check_image_url_thumbnail'].values[0]
    fact_check_date = summary_row['fact_check_date'].values[0]

    data = {
        'fact_check_id': fact_check_id,
        'fact_check_url': fact_check_url,
        'fact_check_explanation_title': fact_check_explanation_title,
        'fact_check_explanation_summary': fact_check_explanation_summary,
        'fact_check_image_url_thumbnail': fact_check_image_url_thumbnail,
        'fact_check_date': fact_check_date}

    claim_data = {

    }
    claimes_storage = []
    quotes_storage = []

    if fact_check_id not in dump_data:
        fact_check_page_path = f"{result_path_fullfact_partial_fact_checks}{row['fact_check_id']}.html"
        if os.path.exists(fact_check_page_path):
            
            with open(fact_check_page_path, encoding='utf-8') as fp:
                page = BeautifulSoup(fp, 'html.parser')
                article = page.find('article')
##TITOLO
                h1_title = article.find('h1', {"class": "highlight-js"}).text.strip()
                data['fact_check_explanation_title'] = h1_title

##DATA PUBBLICAZIONE NOTIZIA
                publish_date = article.find('div', {"class":"published-at"}).text.strip()
                publish_date = find_date_string(publish_date)
                if publish_date is not None:
                    date_object = datetime.strptime(publish_date, '%Y-%m-%d %H:%M:%S')
                    publish_date = date_object.strftime("%Y-%m-%d")
                data['featured_statement_date'] = publish_date
                data['featured_statement_date_year'] = publish_date.split('-')[0]
                data['featured_statement_date_month'] = publish_date.split('-')[1]
                data['featured_statement_date_day'] = publish_date.split('-')[2]

##RECLAMI CORRELATI ALLA NOTIZIA
                claims_cards = article.findAll('div',{"class":"claimConclusion-js"})
                for cards in claims_cards:
                    claim_body = cards.find('div', {"class": "card-body-text"})
                    claims = claim_body.findAll('div', {"class": "col-md-6"})
                    what_was_claimed = claim_body.findAll('div', {"class": "card-claim-body"})
                    our_verdict = claim_body.findAll('div', {"class": "card-conclusion-body"})

                    for idx, claim in enumerate(what_was_claimed, start = 0):  # Aggiunta di enumerate per ottenere sia l'indice che l'elemento
                        claim_data = {}
                        initial_statement = what_was_claimed[idx].find("p", {'class': 'card-text'})
                        claim_data['featured_statement_text'] = initial_statement.text.strip().encode('utf-8', 'ignore').decode('utf-8')
                        claim_data['featured_statement_word_count'] =   len(claim_data['featured_statement_text'].split())
                        hasVerdict = False
                        if our_verdict[idx] is not None:
                            hasVerdict = True
                            statement_debunking = our_verdict[idx].find("p").text.strip().encode('utf-8', 'ignore').decode('utf-8')
                            claim_data['statement_debunking_text'] = statement_debunking
                            claim_data['statement_debunking_word_count'] =   len(claim_data['statement_debunking_text'].split())
                            claim_data['has_verdict'] = hasVerdict
                        claimes_storage.append(claim_data)
                data['claims'] = claimes_storage
                div_article_content = article.find('div', {"class": 'cms-content'})
                if div_article_content is None:
                    div_article_content = article

                ##AFFERMAZIONI CORRELATE ALLA NOTIZIA
                quotes = div_article_content.findAll('blockquote')
                quotes_storage = []
                if quotes:
                  for quote in quotes:
                      quotes_data = {}
                      cites =  quote.findAll("cite")
                      for cite in cites:
                        
                        if len(cite.text) > 0:
                            cite_text = cite.text.strip().encode('utf-8', 'ignore').decode('utf-8')
                            parts = [part.strip() for part in cite_text.split(',')]
                            link = ""
                            if quote.a is not None and 'href' in quote.a.attrs:
                                link = quote.a['href']
                            speaker = parts[0]
                            statement_date = find_date_string(cite_text)
                            statement = quote.find("p").text.strip().encode('utf-8', 'ignore').decode('utf-8')

                            quotes_data['statement'] = statement
                            quotes_data['speaker'] = speaker
                            quotes_data['resource_url'] = link
                            quotes_data['statement_date'] = statement_date
                            quotes_storage.append(quotes_data)

                data['quotes'] = quotes_storage

                ##CONTENUTO ARTICOLO
                div_article_text = div_article_content.findAll('p')
                explanation=""
                links_explanation = []
                for paragraph in div_article_text:
                    explanation += " \n"+ paragraph.text
                    div_article_text_links = paragraph.findAll('a')
                    for link_explanation in div_article_text_links:
                      link = f"{link_explanation.text.strip()};;;{link_explanation.get('href')}"
                      links_explanation.append(link)
                data['fact_check_explanation_full_links'] = ":::".join(links_explanation)
                data['fact_check_explanation_full_text'] = explanation

##ESTRAZIONE GROUND TRUTH
                truthfulness = ''
                truthfulness_motivation = ''
                statement_truthfulness_info = article.find('section', {"class": 'card'})
                if statement_truthfulness_info:
                    truthfulness_container = statement_truthfulness_info.find('p', {"class": 'm-0'})
                    substring = "rated this claim as"
                    if substring in truthfulness_container.text:
                        contenuto, motivation = extract_truthfulness_and_motivation(str(truthfulness_container))
                        if contenuto:
                            truthfulness = contenuto
                        if motivation:
                            truthfulness_motivation = motivation

                data['fact_check_ground_truth_label'] = truthfulness
                data['fact_check_ground_truth_value'] = pol_labels_map.get(truthfulness, '')


##AUTORI ARTICOLO
                author_info = article.find('section', {"class": 'social-media'})
                citation_list = author_info.find('ul', {"class": 'citation'})
                if citation_list is not None:
                    authors = citation_list.findAll('cite')
                    authorList = [author.text for author in authors]
                    data['fact_checker_name'] = ','.join(authorList)

##FATTI CORRELATI
                related_fact_checks = page.find('section', {'class': 'related-factchecks'})
                if related_fact_checks:
                    related_facts = related_fact_checks.findAll('a')
                    data['fact_check_sources'] = ';;;'.join(['{};;;{}'.format(related.text, 'https://fullfact.org' + related.get('href')) for related in related_facts])

##CONTROLLO EVENTUALE CHECK PRECEDENTE
                corrected = any(corrections_df['fact_check_id'] == fact_check_id)
                data['fact_checker_corrected'] = corrected
                if corrected:
                    summary_row = corrections_df[corrections_df['fact_check_id'] == fact_check_id]
                    correction_description = summary_row['correction_description'].values[0]
                    fact_checker_correction_date = summary_row['fact_checker_correction_date'].values[0]        
                    data['fact_checker_correct_description'] = correction_description
                    data['fact_checker_correction_date'] = fact_checker_correction_date
                else:
                    data['fact_checker_correct_description'] = ""
                    data['fact_checker_correction_date'] = ""
                                   
                dump_data[row['fact_check_id']] = data

    serialize_json(result_path_fullfact, f"{scrape_identifier}_fact_checks.json", dump_data)

100%|██████████| 6140/6140 [33:32<00:00,  3.05it/s]


In [8]:
values = set(dump_data[row['fact_check_id']]['fact_check_ground_truth_label'] for row in dump_data.values())
for value in values:
    print(value)


opinion
altered
prank generator
mixture
satire
partly false
true
false
false headline
missing context
